## Populate an RDF database

This notebook reports the main steps to download CSV files, process them and create an RDF dataset from them accordingly to an ontology. 

To measure execution time in Jupyter notebooks: <code>pip install ipython-autotime</code>

In [1]:
# required libraries
import pandas as pd
import os
from pathlib import Path

In [2]:
# parameters and URLs
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
moviesUrl = path + '/data/imdb/IMDb_movies.csv'
namesUrl = path + '/data/imdb/IMDb_names.csv'
joinTableUrl = path + '/data/imdb/IMDb_title_principals.csv'
oscarsUrl = path + '/data/imdb/the_oscar_award.csv'

# country codes
countriesURL = path + '/data/countryCodes/wikipedia-iso-country-codes.csv'

# saving folder
savePath =  path + '/data/rdf/'

## Movies

In [3]:
# Load the CSV files in memory
movies = pd.read_csv(moviesUrl, sep=',', index_col='imdb_title_id')
# cast year to int. If type(year) = str --> Literal= year-01-01
movies.astype({'year': 'int32'}).dtypes

title                     object
original_title            object
year                       int32
date_published            object
genre                     object
duration                   int64
country                   object
language                  object
director                  object
writer                    object
production_company        object
actors                    object
description               object
avg_vote                 float64
votes                      int64
budget                    object
usa_gross_income          object
worlwide_gross_income     object
metascore                float64
reviews_from_users       float64
reviews_from_critics     float64
dtype: object

In [4]:
#load the country codes
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
countries = pd.read_csv(countriesURL, sep=',', index_col='Name', keep_default_na=False, na_values=['_'])

In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85855 entries, tt0000009 to tt9914942
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  85855 non-null  object 
 1   original_title         85855 non-null  object 
 2   year                   85855 non-null  int64  
 3   date_published         85855 non-null  object 
 4   genre                  85855 non-null  object 
 5   duration               85855 non-null  int64  
 6   country                85791 non-null  object 
 7   language               85022 non-null  object 
 8   director               85768 non-null  object 
 9   writer                 84283 non-null  object 
 10  production_company     81400 non-null  object 
 11  actors                 85786 non-null  object 
 12  description            83740 non-null  object 
 13  avg_vote               85855 non-null  float64
 14  votes                  85855 non-null  int64  


We need to install <code>RDFLib</code>

<code>pip3 install rdflib </code> [Documentation](https://rdflib.readthedocs.io/en/stable/gettingstarted.html)

In [6]:
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD



In [7]:
# Construct the country and the movie ontology namespaces not known by RDFlib
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
MO = Namespace("http://www.dei.unipd.it/database2/movieOntology#")

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("mo", MO)


In [8]:
# CHECK DATE 
import datetime


In [32]:
%%time 
#measure execution time

#iterate over the movies dataframe
for index, row in movies.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    Movie = URIRef(MO[index])
    # Add triples using store's add() method.
    g.add((Movie, RDF.type, MO.Movie))
    g.add((Movie, MO['originalTitle'], Literal(row['original_title'], datatype=XSD.string)))
    g.add((Movie, MO['alternativeTitle'], Literal(row['title'], datatype=XSD.string)))
    g.add((Movie, MO['releaseYear'], Literal(row['year'], datatype=XSD.gYear)))
   
    try:
        datetime.datetime.strptime(str(row['date_published']), '%Y-%m-%d')
        g.add((Movie, MO['releaseDate'], Literal(row['date_published'], datatype=XSD.date)))
    except ValueError:
        # probably it's the year alone
        # check length
        if (len(row['date_published'])==4):
            #it is a year
            g.add((Movie, MO['releaseDate'], Literal(row['date_published']+"-01-01", datatype=XSD.date)))

    g.add((Movie, MO['duration'], Literal(row['duration'], datatype=XSD.integer)))

    ## how to handle genre
    #there can be more than one genre per movie
    for gN in row['genre'].split(','):
        Genre = URIRef(MO[gN.strip()])
        g.add((Movie, MO['hasGenre'], Genre))    

    ## handle country
    #there can be more than one country per movie
    for c in str(row['country']).split(','):
        cName = c.strip()
        # check if the country exists
        # country.index == x returns an array of booleans, thus we need to use the any() method
        if((countries.index == cName).any() == True):
            #get the country code, convert to string and get the lower case to match the country codes in the ontology 
            code = str(countries[countries.index == cName]['Alpha-2 code'][0]).lower()
            # create the RDF node
            Country = URIRef(CNS[code])
            # add the edge connecting the Movie and the Country 
            g.add((Movie, MO['hasCountry'], Country))    




CPU times: user 1min 26s, sys: 1.47 s, total: 1min 27s
Wall time: 1min 29s


In [33]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'movies.ttl', 'w') as file:
    file.write(g.serialize(format='turtle').decode("utf-8"))


--- saving serialization ---
CPU times: user 1min 3s, sys: 1.04 s, total: 1min 4s
Wall time: 1min 6s


# Referential integrity
Note that in RDF we are in an open world situation. We cannot guarantee the referential integrity between the entities. 

## Person

Let us generate the RDF data relative to the movie workers.

In [34]:
# Load the CSV files in memory
people = pd.read_csv(namesUrl, sep=',', index_col='imdb_name_id', keep_default_na=False, na_values=['_'])

In [35]:
people.info()

<class 'pandas.core.frame.DataFrame'>
Index: 297705 entries, nm0000001 to nm3834123
Data columns (total 16 columns):
name                     297705 non-null object
birth_name               297705 non-null object
height                   297705 non-null object
bio                      297705 non-null object
birth_details            297705 non-null object
date_of_birth            297705 non-null object
place_of_birth           297705 non-null object
death_details            297705 non-null object
date_of_death            297705 non-null object
place_of_death           297705 non-null object
reason_of_death          297705 non-null object
spouses_string           297705 non-null object
spouses                  297705 non-null int64
divorces                 297705 non-null int64
spouses_with_children    297705 non-null int64
children                 297705 non-null int64
dtypes: int64(4), object(12)
memory usage: 38.6+ MB


People are modeled with the FOAF ontology. 
Refer to [FOAF Documentation](http://xmlns.com/foaf/spec/)

In [36]:
#create a new graph
g = Graph()

In [37]:
%%time 
#measure execution time

#iterate over the person dataframe
for index, row in people.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the person id as URI
    Person = URIRef(MO[index])
    g.add((Person, RDF.type, FOAF.Person))
    # Add triples using store's add() method.
    g.add((Person, FOAF['name'], Literal(row['name'], datatype=XSD.string)))
    if row['date_of_birth'] != '':
        try:
            datetime.datetime.strptime(str(row['date_of_birth']), '%Y-%m-%d')
            g.add((Person, MO['birthday'], Literal(row['date_of_birth'], datatype=XSD.date)))
        except ValueError:
            # probably it's the year alone
            # check length
            if (len(row['date_of_birth'])==4):
                #it is a year
                g.add((Person, MO['birthday'], Literal(row['date_of_birth']+"-01-01", datatype=XSD.date)))
    
    if row['place_of_birth'] != '':
        g.add((Person, MO['birthplace'], Literal(row['place_of_birth'], datatype=XSD.string)))
    
    # check if the death day is not empty--i.e., the person is still alive
    if row['date_of_death'] != '':
        try:
            datetime.datetime.strptime(str(row['date_of_death']), '%Y-%m-%d')
            g.add((Person, MO['deathDay'], Literal(row['date_of_death'], datatype=XSD.date)))
        except ValueError:
            # probably it's the year alone
            # check length
            if (len(row['date_of_death'])==4):
                #it is a year
                g.add((Person, MO['deathDay'], Literal(row['date_of_death']+"-01-01", datatype=XSD.date)))
        
# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("mo", MO)

CPU times: user 1min 25s, sys: 1.43 s, total: 1min 27s
Wall time: 1min 27s


In [38]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'names.ttl', 'w') as file:
    file.write(g.serialize(format='turtle').decode("utf-8"))

--- saving serialization ---
CPU times: user 1min 19s, sys: 1.31 s, total: 1min 20s
Wall time: 1min 21s


## Person-Movie Join

In [39]:
# Load the CSV files in memory
join = pd.read_csv(joinTableUrl, sep=',', index_col='imdb_title_id', keep_default_na=False, na_values=['_'])

In [40]:
#create a new graph
g = Graph()

In [41]:
#regular expressions
import re
actor = re.compile('act*')

In [42]:
%%time 
#measure execution time

#iterate over the join table dataframe
for index, row in join.iterrows():
    # Create the node about the movie
    # note that we do not add this resource to the database (created before)
    Movie = URIRef(MO[index])
    
    # Create the node about the person
    # note that we do not add this resource to the database (created before)
    Person = URIRef(MO[row['imdb_name_id']])
    # get the role of the person
    role = row['category']
    
    # we have an actor or actress
    if actor.match(role): 
        g.add((Person, MO['acted'], Movie))
    elif (role=='director'):
        g.add((Person, MO['directed'], Movie))
    else:
        # note that, with the defined ontology, we cannot caracterize the specific role of this person in the movie. 
        # why?
        g.add((Person, MO['worked'], Movie))

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("mo", MO)

CPU times: user 2min 21s, sys: 2.18 s, total: 2min 24s
Wall time: 2min 27s


In [43]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'name_movie_join.ttl', 'w') as file:
    file.write(g.serialize(format='turtle').decode("utf-8"))

--- saving serialization ---
CPU times: user 1min 20s, sys: 1.57 s, total: 1min 22s
Wall time: 1min 23s


## Awards - Oscars data
Note that if we do not check the referential integrity then we could produce ghost triple movie-nominee-oscar where the movie is not in the RDF graph.

On the other hand, we can check if an actor or a movie exists by using the DataFrame in Python. Note that this is an external check and not a constraints met by the RDF DB.



In [44]:
# Load the CSV files in memory
oscars = pd.read_csv(oscarsUrl, sep=',', keep_default_na=False, na_values=['_'])

In [45]:
from num2words import num2words
import string
import re
#create a new graph
g = Graph()

In [46]:
%%time
#iterate over the join table dataframe
for index,row in oscars.iterrows():
    #create the oscar with a custom id 
    cat = re.sub(r'[^\w\s]','',row['category'])
    Oscar = URIRef(MO['oscar_'+cat.replace(" ", "").lower()+'_'+ str(num2words(row['ceremony'], to='ordinal'))])
    
    # check if there already is at least a triple about this oscar
    if not (Oscar, None, None) in g:    
        # check if the oscar is already in the graph
        g.add((Oscar, RDF.type, MO.Oscar))
        g.add((Oscar, MO['category'], Literal(row['category'].lower(), datatype=XSD.string)))
        g.add((Oscar, MO['year'], Literal(row['year_ceremony'], datatype=XSD.gYear)))
    
    # check if there is a name matching the people, meaning that the oscar can be associated to a person
    if (people["name"] == row["name"]).any() == True :
        #there is a person with this name
        # Create the node about the person
        # note that we do not add this resource to the database (created before)
        Person = URIRef(MO[people[people["name"]==row["name"]].index[0]])
        if row['winner']:
            g.add((Person, MO['winner'], Oscar))
        else:
            g.add((Person, MO['nominated'], Oscar))
    
    # an oscar for a person is also to be considered an oscar for the movie
    # check if the movie is in our DB
    if (movies["original_title"] == row["film"]).any():
        # there is a movie with this title
        Movie = URIRef(MO[movies[movies["original_title"]==row["film"]].index[0]])
        if row['winner']:
            g.add((Movie, MO['winner'], Oscar))
        else:
            g.add((Movie, MO['nominated'], Oscar))

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("mo", MO)

CPU times: user 4min 53s, sys: 4.85 s, total: 4min 58s
Wall time: 5min 8s


In [47]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'oscars.ttl', 'w') as file:
    file.write(g.serialize(format='turtle').decode("utf-8"))

--- saving serialization ---
CPU times: user 2.88 s, sys: 60.9 ms, total: 2.94 s
Wall time: 4.3 s
